In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import * 

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@adlsdatabrickseteproject.dfs.core.windows.net/products")

In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete_project_catalog.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE  
LANGUAGE SQL 
RETURN p_price * 0.90

In [0]:
%sql
select product_id, price ,databricks_ete_project_catalog.bronze.discount_func(price) as discounted_price
FROM products

In [0]:
df = df.withColumn("discounted_price",expr("databricks_ete_project_catalog.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete_project_catalog.bronze.upper_func(p_brand STRING)
RETURNS STRING 
LANGUAGE PYTHON 
AS 
$$ 
    return p_brand.upper()
$$

In [0]:
%sql
SELECT product_id, brand, databricks_ete_project_catalog.bronze.upper_func(brand) as brand_upper
FROM products

In [0]:
df.write.format("delta")\
            .mode("append")\
            .option("path","abfss://silver@adlsdatabrickseteproject.dfs.core.windows.net/products")\
            .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_ete_project_catalog.silver.products_silver 
USING DELTA 
LOCATION 'abfss://silver@adlsdatabrickseteproject.dfs.core.windows.net/products'